In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType, StringType, TimestampType
import boto3
import pandas as pd
import gc
!pip install s3fs

    100% |████████████████████████████████| 5.9MB 5.1MB/s eta 0:00:01    52% |████████████████▊               | 3.1MB 28.3MB/s eta 0:00:01    95% |██████████████████████████████▋ | 5.6MB 29.4MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 11.7MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 21.9MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 14.8MB/s ta 0:00:01
awscli 1.16.17 has requirement botocore==1.12.7, but you'll have botocore 1.14.9 which is incompatible.
awscli 1.16.17 has requirement s3transfer<0.2.0,>=0.1.12, but you'll have s3transfer 0.3.2 which is incompatible.
  Found existing installation: botocore 1.12.7
    Uninstalling botocore-1.12.7:
      Successfully uninstalled botocore-1.12.7
  Found existing installation: s3transfer 0.1.13
    Uninstalling s3transfer-0.1.13:
      Successfully uninstalled s3transfer-0.1.13
  Found existing installation: boto3 1.9.7
    Uninstalling boto3-1.9.7:
      Successfully uninstalled 

In [2]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

In [3]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

In [4]:
spark = create_spark_session()

In [5]:
s3 = boto3.client("s3", region_name="us-west-2", aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
                    aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'])

In [6]:
searchobj = "log_data"
lcobj = list(s3.list_objects_v2(Bucket="udacity-dend", 
                                Prefix=searchobj + "/").values())

In [7]:
log_data_list = []
for k in lcobj[2]:
    if k["Key"].find(".json") > -1:
        log_data_list.append("s3a://udacity-dend/" + k["Key"])

In [8]:
searchobj = "song_data"
lcobj = list(s3.list_objects_v2(Bucket="udacity-dend", 
                                Prefix=searchobj + "/").values())

In [9]:
song_data_list = []
for k in lcobj[2]:
    if k["Key"].find(".json") > -1:
        song_data_list.append("s3a://udacity-dend/" + k["Key"])

In [10]:
#df = pd.read_json(log_data_list[0], lines=True)

In [14]:
%%time
sdf = spark.read.json(log_data_list)

CPU times: user 10.7 ms, sys: 0 ns, total: 10.7 ms
Wall time: 22.2 s


In [15]:
#print(sdf.count())
#assert df.shape[1] == len(sdf.columns)

In [16]:
#sdf.limit(5).toPandas()

In [17]:
sdf = sdf.filter(F.col("page")=="NextSong")

In [18]:
userdf = sdf.select(F.col("userId").alias("user_id"), 
                 F.col("firstname").alias("first_name"), 
                 F.col("lastname").alias("last_name"), 
                 "gender", "level").distinct()\
         .orderBy("userId")

In [19]:
#userdf.count()

In [20]:
#userdf.limit(5).toPandas()

In [18]:
%%time
userdf.write.mode("overwrite").parquet("s3a://christophndde4/user_table/")

CPU times: user 82.8 ms, sys: 24.5 ms, total: 107 ms
Wall time: 10min 39s


In [21]:
gc.collect()

161

In [22]:
sdf = sdf.withColumn("timestamp", F.expr("cast(ts / 1000 as timestamp)"))

In [23]:
sdf = sdf.withColumn("datetime", F.expr("cast(timestamp as date)"))

In [24]:
#sdf.limit(3).toPandas()

In [25]:
sdf.createOrReplaceTempView("sdftab")

In [26]:
tdf = spark.sql("""
    SELECT DISTINCT
    timestamp AS start_time,
    HOUR(timestamp) AS hour,
    DAY(timestamp) AS day,
    WEEKOFYEAR(timestamp) as week,
    MONTH(timestamp) as month,
    YEAR(timestamp) as year,
    DAYOFWEEK(timestamp) as weekday
    FROM
    sdftab
    ORDER BY 1
    """)

In [25]:
%%time
tdf.write.mode("overwrite").parquet("s3a://christophndde4/time_table/")

CPU times: user 168 ms, sys: 44.8 ms, total: 213 ms
Wall time: 22min 4s


In [27]:
gc.collect()

30

In [28]:
#tdf.count()

In [29]:
#tdf.printSchema()

In [30]:
#tdf.limit(5).toPandas()

In [31]:
%%time
songstage_df = spark.read.json(song_data_list)

CPU times: user 153 ms, sys: 17.7 ms, total: 171 ms
Wall time: 6min 24s


In [32]:
#songstage_df.count()

In [33]:
#songstage_df.printSchema()

In [34]:
songsdf = songstage_df.select("song_id", "title", "artist_id", "year", 
                              "duration")\
                      .orderBy(F.col("song_id"))

In [32]:
%%time
songsdf.write.mode("overwrite").parquet("s3a://christophndde4/song_table/")

CPU times: user 193 ms, sys: 52.2 ms, total: 245 ms
Wall time: 26min 31s


In [35]:
#songsdf.count()

In [36]:
#songsdf.printSchema()

In [37]:
#songsdf.limit(5).toPandas()

In [38]:
gc.collect()

107

In [39]:
artistdf = songstage_df.select("artist_id", 
                               F.col("artist_name").alias("name"),
                               F.col("artist_location").alias("location"),
                               F.col("artist_latitude").alias("latitude"),
                               F.col("artist_longitude").alias("longitude"))\
                       .distinct().orderBy(F.col("artist_id"))

In [31]:
%%time
artistdf.write.mode("overwrite").parquet("s3a://christophndde4/artist_table/")

CPU times: user 270 ms, sys: 54 ms, total: 324 ms
Wall time: 24min 28s


In [40]:
#artistdf.count()

In [41]:
#artistdf.printSchema()

In [42]:
#artistdf.limit(5).toPandas()

In [43]:
gc.collect()

118

In [44]:
songstage_df.limit(3).toPandas()
sdf.limit(3).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,timestamp,datetime
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:30:26.796,2018-11-15
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:41:21.796,2018-11-15
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:45:41.796,2018-11-15


In [45]:
sdf = sdf.withColumn("songplay_id", F.monotonically_increasing_id())

In [46]:
sdf.createOrReplaceTempView("sdftab")
artistdf.createOrReplaceTempView("artisttab")
songsdf.createOrReplaceTempView("songstab")

In [47]:
songplay_df = spark.sql("""
    SELECT 
    s.songplay_id,
    s.timestamp AS start_time,
    s.userId as user_id,
    s.level,
    so.song_id,
    a.artist_id,
    s.sessionId AS session_id,
    a.location,
    s.userAgent AS user_agent
    FROM
    sdftab s
    LEFT JOIN artisttab a
    ON a.name = s.artist
    LEFT JOIN songstab so
    ON so.artist_id = a.artist_id
    AND so.title = s.song
    WHERE a.artist_id IS NOT NULL
    AND so.song_id IS NOT NULL
    LIMIT 5""")

In [48]:
songplay_df.toPandas()

Py4JJavaError: An error occurred while calling o113.collectToPython.
: org.apache.spark.SparkException: Could not execute broadcast in 300 secs. You can increase the timeout for broadcasts via spark.sql.broadcastTimeout or disable broadcast join by setting spark.sql.autoBroadcastJoinThreshold to -1
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:150)
	at org.apache.spark.sql.execution.InputAdapter.doExecuteBroadcast(WholeStageCodegenExec.scala:387)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:144)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:140)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:152)
	at org.apache.spark.sql